In [ ]:
import pandas as pd
import json
PATH = "csse_covid_19_daily_reports/"

In [ ]:
with open('csse_covid_19_data/country_convert.json','r',encoding='utf-8-sig') as json_file:
  json_data = json.load(json_file)

def country_name_convert(row):
  if row['Country_Region'] in json_data:
    return json_data[row['Country_Region']]
  return row['Country_Region']

def create_dateframe(filename):
  doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')  # 1. csv 파일 읽기
  try:
    doc = doc[['Country_Region','Confirmed']]   # 2. 특정 컬럼만 선택해서 데이터프레임 만들기
  except:
    doc = doc[['Country/Region','Confirmed']]
    doc.columns = ['Country_Region','Confirmed']
  
  doc = doc.dropna(subset=['Confirmed'])        # 3. 특정 컬럼에 없는 데이터 삭제하기
  doc['Country_Region'] = doc.apply(country_name_convert, axis=1) # 4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경
  doc = doc.astype({'Confirmed':'int64'})  # 5. 특정 컬럼의 데이터타입 변경하기
  doc = doc.groupby('Country_Region').sum()     # 6. 특정 컬럼으로 중복된 데이터를 합치기

  # 7. 파일명을 기반으로 날짜 문자열 변환하고, 'Confirmed' 컬럼명 변경하기
  date_column = filename.split('.')[0].lstrip('0').replace('-','/')
  doc.columns = [date_column]
  return doc

In [ ]:
import os

def generate_dateframe_by_path(PATH):
  file_list, csv_list = os.listdir(PATH), list()
  first_doc = True
  for file in file_list:
    if file.split('.')[-1] == 'csv':
      csv_list.append(file)
  csv_list.sort()
  
  temp_list = []
  for csv in csv_list[:]:
    if '2021' in csv:
      temp_list.append(csv)
      csv_list.remove(csv)

  csv_list.extend(temp_list)

  for file in csv_list:
    doc = create_dateframe(file)
    if first_doc:
      final_doc, first_doc = doc, False
    else:
      final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)

  final_doc = final_doc.fillna(0)
  return final_doc

In [ ]:
doc = generate_dateframe_by_path(PATH)
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/01/2020,...,1/02/2021,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021
Country_Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,51526.0,51526.0,53011.0,53105.0,53105.0,53207.0,53332.0,53400.0,53489.0,53538.0,53584.0,53584.0,53775.0,53831.0,53938.0,53984.0,54062.0,54141.0,54278.0,54403.0,54483.0,54559.0,54595.0,54672.0,54750.0,54854.0,54891.0,54939.0,55008.0,55023.0,55059.0,55121.0,55174.0,55231.0,55265.0,55330.0,55335.0,55359.0,55384.0,55402.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,58991.0,59438.0,59623.0,60283.0,61008.0,61705.0,62378.0,63033.0,63595.0,63971.0,64627.0,65334.0,65994.0,66635.0,67216.0,67690.0,67982.0,68568.0,69238.0,69916.0,70655.0,71441.0,72274.0,72812.0,73691.0,74567.0,75454.0,76350.0,77251.0,78127.0,78992.0,79934.0,80941.0,81993.0,83082.0,84212.0,85336.0,86289.0,87528.0,88671.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,100159.0,100408.0,100645.0,100873.0,101120.0,101382.0,101657.0,101913.0,102144.0,102369.0,102641.0,102860.0,103127.0,103381.0,103611.0,103833.0,104092.0,104341.0,104606.0,104852.0,105124.0,105369.0,105596.0,105854.0,106097.0,106359.0,106610.0,106887.0,107122.0,107339.0,107578.0,107841.0,108116.0,108381.0,108629.0,108629.0,109088.0,109313.0,109559.0,109782.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8166.0,8192.0,8249.0,8308.0,8348.0,8348.0,8489.0,8586.0,8586.0,8586.0,8682.0,8818.0,8868.0,8946.0,9038.0,9083.0,9083.0,9194.0,9308.0,9379.0,9416.0,9499.0,9549.0,9596.0,9638.0,9716.0,9779.0,9837.0,9885.0,9937.0,9972.0,10017.0,10070.0,10137.0,10172.0,10206.0,10251.0,10275.0,10312.0,10352.0
Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17608.0,17642.0,17684.0,17756.0,17864.0,17974.0,18066.0,18156.0,18193.0,18254.0,18343.0,18425.0,18613.0,18679.0,18765.0,18875.0,18926.0,19011.0,19093.0,19177.0,19269.0,19367.0,19399.0,19476.0,19553.0,19580.0,19672.0,19723.0,19782.0,19796.0,19829.0,19900.0,19937.0,19996.0,20030.0,20062.0,20086.0,20112.0,20163.0,20210.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,6.0,6.0,8.0,8.0,8.0,10.0,10.0,13.0,13.0,14.0,15.0,15.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,...,1482.0,1494.0,1497.0,1504.0,1505.0,1509.0,1512.0,1513.0,1514.0,1515.0,1520.0,1521.0,1531.0,1536.0,1537

In [ ]:
doc = doc.astype('int64')
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/01/2020,...,1/02/2021,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021
Country_Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,...,51526,51526,53011,53105,53105,53207,53332,53400,53489,53538,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403,54483,54559,54595,54672,54750,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265,55330,55335,55359,55384,55402
Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,58991,59438,59623,60283,61008,61705,62378,63033,63595,63971,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916,70655,71441,72274,72812,73691,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082,84212,85336,86289,87528,88671
Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,...,100159,100408,100645,100873,101120,101382,101657,101913,102144,102369,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852,105124,105369,105596,105854,106097,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629,108629,109088,109313,109559,109782
Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8166,8192,8249,8308,8348,8348,8489,8586,8586,8586,8682,8818,8868,8946,9038,9083,9083,9194,9308,9379,9416,9499,9549,9596,9638,9716,9779,9837,9885,9937,9972,10017,10070,10137,10172,10206,10251,10275,10312,10352
Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17608,17642,17684,17756,17864,17974,18066,18156,18193,18254,18343,18425,18613,18679,18765,18875,18926,19011,19093,19177,19269,19367,19399,19476,19553,19580,19672,19723,19782,19796,19829,19900,19937,19996,20030,20062,20086,20112,20163,20210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,0,2,2,2,2,2,2,2,2,2,6,6,8,8,8,10,10,13,13,14,15,15,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,...,1482,1494,1497,1504,1505,1509,1512,1513,1514,1515,1520,1521,1531,1536,1537,1537,1539,1540,1544,1546,1548,1548,1548,1549,1551,1553,1651,1657,1767,1817,1850,1882,1948,1957,1976,1985,2001,2050,2064,2091
West Bank and Gaza,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,140287,141219,142228,143169,144257,145252,146074,146701,147400,148171,148968,149769,150505,151142,151569,152031,152555,153093,153590,154063,154557,155006,155414,155884,156393,156996,157593,158168,158559,158962,159443,159956,160426,161087,161559,162029,162601,163213,163975,164736
Yemen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2101,2101,2101,2101,2102,2104,2104,2104,2104,2105,2107,2109,2110,2111,2112,2112,2113,2115,2115,2115,2118,2118,2118,2118,2119,2120,2120,2120,2120,2121,2122,2122,2122,2122,2124,2127,2127,2131,2131,2133


In [ ]:
doc.to_csv('final_confirmed_df.csv')

In [ ]:
import pandas as pd
df_confirmed = pd.read_csv('final_confirmed_df.csv')
df_confirmed.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,...,1/02/2021,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021
0,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,...,51526,51526,53011,53105,53105,53207,53332,53400,53489,53538,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403,54483,54559,54595,54672,54750,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265,55330,55335,55359,55384,55402
1,Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,58991,59438,59623,60283,61008,61705,62378,63033,63595,63971,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916,70655,71441,72274,72812,73691,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082,84212,85336,86289,87528,88671
2,Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,100159,100408,100645,100873,101120,101382,101657,101913,102144,102369,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852,105124,105369,105596,105854,106097,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629,108629,109088,109313,109559,109782
3,Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8166,8192,8249,8308,8348,8348,8489,8586,8586,8586,8682,8818,8868,8946,9038,9083,9083,9194,9308,9379,9416,9499,9549,9596,9638,9716,9779,9837,9885,9937,9972,10017,10070,10137,10172,10206,10251,10275,10312,10352
4,Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17608,17642,17684,17756,17864,17974,18066,18156,18193,18254,18343,18425,18613,18679,18765,18875,18926,19011,19093,19177,19269,19367,19399,19476,19553,19580,19672,19723,19782,19796,19829,19900,19937,19996,20030,20062,20086,20112,20163,20210


In [ ]:
df_confirmed.shape

(192, 387)

In [ ]:
country_info = pd.read_csv('csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv',encoding='utf-8-sig',keep_default_na=False, na_values='')
country_info.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.20270,17.873900,Angola,32866268.0


In [ ]:
country_info = country_info[['iso2','Country_Region']]
country_info.head()

,iso2,Country_Region
0,AF,Afghanistan
1,AL,Albania
2,DZ,Algeria
3,AD,Andorra
4,AO,Angola


In [ ]:
country_info.shape

(4173, 2)

In [ ]:
country_info = country_info.drop_duplicates(subset='Country_Region', keep='first')
country_info.shape

(194, 2)

In [ ]:
doc_final_country = pd.merge(df_confirmed, country_info, how='left',on='Country_Region')
doc_final_country.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,...,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,...,51526,53011,53105,53105,53207,53332,53400,53489,53538,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403,54483,54559,54595,54672,54750,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265,55330,55335,55359,55384,55402,AF
1,Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,59438,59623,60283,61008,61705,62378,63033,63595,63971,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916,70655,71441,72274,72812,73691,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082,84212,85336,86289,87528,88671,AL
2,Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,100408,100645,100873,101120,101382,101657,101913,102144,102369,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852,105124,105369,105596,105854,106097,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629,108629,109088,109313,109559,109782,DZ
3,Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8192,8249,8308,8348,8348,8489,8586,8586,8586,8682,8818,8868,8946,9038,9083,9083,9194,9308,9379,9416,9499,9549,9596,9638,9716,9779,9837,9885,9937,9972,10017,10070,10137,10172,10206,10251,10275,10312,10352,AD
4,Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17642,17684,17756,17864,17974,18066,18156,18193,18254,18343,18425,18613,18679,18765,18875,18926,19011,19093,19177,19269,19367,19399,19476,19553,19580,19672,19723,19782,19796,19829,19900,19937,19996,20030,20062,20086,20112,20163,20210,AO


In [ ]:
doc_final_country.shape

(192, 388)

In [ ]:
doc_final_country.isnull().sum()

Country_Region    0
1/22/2020         0
1/23/2020         0
1/24/2020         0
1/25/2020         0
                 ..
2/07/2021         0
2/08/2021         0
2/09/2021         0
2/10/2021         0
iso2              5
Length: 388, dtype: int64

In [ ]:
doc_final_country[doc_final_country['iso2'].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,...,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021,iso2
44,Cruise Ship,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
49,Diamond Princess,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,NaN
105,MS Zaandam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,NaN
132,Others,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,61,64,135,135,175,175,218,285,355,454,542,621,634,634,634,691,691,691,705,705,705,705,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
169,Taiwan,1,1,3,3,4,5,8,8,9,10,10,10,10,11,11,16,16,17,18,18,18,18,18,18,18,20,22,22,23,24,26,26,28,30,31,32,32,34,39,...,812,815,817,819,822,825,828,828,834,838,842,842,843,851,855,862,868,870,881,881,884,889,889,890,893,895,899,909,911,912,915,917,919,923,924,927,928,933,935,NaN


In [ ]:
doc_final_country = doc_final_country.dropna(subset=['iso2'])

In [ ]:
doc_final_country[doc_final_country['iso2'].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,...,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021,iso2


In [ ]:
def create_flag_link(row):
  flag_link = 'https://www.countryflags.io/' + row + '/flat/64.png'
  return flag_link

doc_final_country['iso2'] = doc_final_country['iso2'].apply(create_flag_link)

In [ ]:
doc_final_country.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,...,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,...,51526,53011,53105,53105,53207,53332,53400,53489,53538,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403,54483,54559,54595,54672,54750,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265,55330,55335,55359,55384,55402,https://www.countryflags.io/AF/flat/64.png
1,Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,59438,59623,60283,61008,61705,62378,63033,63595,63971,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916,70655,71441,72274,72812,73691,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082,84212,85336,86289,87528,88671,https://www.countryflags.io/AL/flat/64.png
2,Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,100408,100645,100873,101120,101382,101657,101913,102144,102369,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852,105124,105369,105596,105854,106097,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629,108629,109088,109313,109559,109782,https://www.countryflags.io/DZ/flat/64.png
3,Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8192,8249,8308,8348,8348,8489,8586,8586,8586,8682,8818,8868,8946,9038,9083,9083,9194,9308,9379,9416,9499,9549,9596,9638,9716,9779,9837,9885,9937,9972,10017,10070,10137,10172,10206,10251,10275,10312,10352,https://www.countryflags.io/AD/flat/64.png
4,Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17642,17684,17756,17864,17974,18066,18156,18193,18254,18343,18425,18613,18679,18765,18875,18926,19011,19093,19177,19269,19367,19399,19476,19553,19580,19672,19723,19782,19796,19829,19900,19937,19996,20030,20062,20086,20112,20163,20210,https://www.countryflags.io/AO/flat/64.png


In [ ]:
cols = doc_final_country.columns.tolist()
cols

['Country_Region',
 '1/22/2020',
 '1/23/2020',
 '1/24/2020',
 '1/25/2020',
 '1/26/2020',
 '1/27/2020',
 '1/28/2020',
 '1/29/2020',
 '1/30/2020',
 '1/31/2020',
 '2/01/2020',
 '2/02/2020',
 '2/03/2020',
 '2/04/2020',
 '2/05/2020',
 '2/06/2020',
 '2/07/2020',
 '2/08/2020',
 '2/09/2020',
 '2/10/2020',
 '2/11/2020',
 '2/12/2020',
 '2/13/2020',
 '2/14/2020',
 '2/15/2020',
 '2/16/2020',
 '2/17/2020',
 '2/18/2020',
 '2/19/2020',
 '2/20/2020',
 '2/21/2020',
 '2/22/2020',
 '2/23/2020',
 '2/24/2020',
 '2/25/2020',
 '2/26/2020',
 '2/27/2020',
 '2/28/2020',
 '2/29/2020',
 '3/01/2020',
 '3/02/2020',
 '3/03/2020',
 '3/04/2020',
 '3/05/2020',
 '3/06/2020',
 '3/07/2020',
 '3/08/2020',
 '3/09/2020',
 '3/10/2020',
 '3/11/2020',
 '3/12/2020',
 '3/13/2020',
 '3/14/2020',
 '3/15/2020',
 '3/16/2020',
 '3/17/2020',
 '3/18/2020',
 '3/19/2020',
 '3/20/2020',
 '3/21/2020',
 '3/22/2020',
 '3/23/2020',
 '3/24/2020',
 '3/25/2020',
 '3/26/2020',
 '3/27/2020',
 '3/28/2020',
 '3/29/2020',
 '3/30/2020',
 '3/31/2020',
 

In [ ]:
cols.remove('iso2')
cols.insert(1, 'iso2')

In [ ]:
doc_final_country = doc_final_country[cols]

In [ ]:
doc_final_country.head()

,Country_Region,iso2,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,...,1/02/2021,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021
0,Afghanistan,https://www.countryflags.io/AF/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,51526,51526,53011,53105,53105,53207,53332,53400,53489,53538,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403,54483,54559,54595,54672,54750,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265,55330,55335,55359,55384,55402
1,Albania,https://www.countryflags.io/AL/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,58991,59438,59623,60283,61008,61705,62378,63033,63595,63971,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916,70655,71441,72274,72812,73691,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082,84212,85336,86289,87528,88671
2,Algeria,https://www.countryflags.io/DZ/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,...,100159,100408,100645,100873,101120,101382,101657,101913,102144,102369,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852,105124,105369,105596,105854,106097,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629,108629,109088,109313,109559,109782
3,Andorra,https://www.countryflags.io/AD/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8166,8192,8249,8308,8348,8348,8489,8586,8586,8586,8682,8818,8868,8946,9038,9083,9083,9194,9308,9379,9416,9499,9549,9596,9638,9716,9779,9837,9885,9937,9972,10017,10070,10137,10172,10206,10251,10275,10312,10352
4,Angola,https://www.countryflags.io/AO/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17608,17642,17684,17756,17864,17974,18066,18156,18193,18254,18343,18425,18613,18679,18765,18875,18926,19011,19093,19177,19269,19367,19399,19476,19553,19580,19672,19723,19782,19796,19829,19900,19937,19996,20030,20062,20086,20112,20163,20210


In [ ]:
cols[1] = 'Country_Flag'

In [ ]:
doc_final_country.columns = cols
doc_final_country.head()

,Country_Region,Country_Flag,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,...,1/02/2021,1/03/2021,1/04/2021,1/05/2021,1/06/2021,1/07/2021,1/08/2021,1/09/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/01/2021,2/02/2021,2/03/2021,2/04/2021,2/05/2021,2/06/2021,2/07/2021,2/08/2021,2/09/2021,2/10/2021
0,Afghanistan,https://www.countryflags.io/AF/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,...,51526,51526,53011,53105,53105,53207,53332,53400,53489,53538,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403,54483,54559,54595,54672,54750,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265,55330,55335,55359,55384,55402
1,Albania,https://www.countryflags.io/AL/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,58991,59438,59623,60283,61008,61705,62378,63033,63595,63971,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916,70655,71441,72274,72812,73691,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082,84212,85336,86289,87528,88671
2,Algeria,https://www.countryflags.io/DZ/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,...,100159,100408,100645,100873,101120,101382,101657,101913,102144,102369,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852,105124,105369,105596,105854,106097,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629,108629,109088,109313,109559,109782
3,Andorra,https://www.countryflags.io/AD/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8166,8192,8249,8308,8348,8348,8489,8586,8586,8586,8682,8818,8868,8946,9038,9083,9083,9194,9308,9379,9416,9499,9549,9596,9638,9716,9779,9837,9885,9937,9972,10017,10070,10137,10172,10206,10251,10275,10312,10352
4,Angola,https://www.countryflags.io/AO/flat/64.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17608,17642,17684,17756,17864,17974,18066,18156,18193,18254,18343,18425,18613,18679,18765,18875,18926,19011,19093,19177,19269,19367,19399,19476,19553,19580,19672,19723,19782,19796,19829,19900,19937,19996,20030,20062,20086,20112,20163,20210


In [ ]:
doc_final_country.to_csv('final_confirmed_for_graph.csv')